In [ ]:
import json
import matplotlib.pyplot as plt

JSON_EXT = '.json'
CYTOSCAPE_JS_EXT = '.cyjs'

Blues = plt.get_cmap('Blues')

def find_min_max(nodes):
    
    scores = []
    
    for node in nodes:
         if "score" in node["data"]:
                scores.append(node["data"]["score"])
    
    min_score = min(scores)
    max_score = max(scores)
    
    return (min_score, max_score)

In [ ]:
def convert2sigma(in_file_name, out_file_name):
    print('Source file: ' + in_file_name)

    # Load Original Cyjs file
    cyjs_file = open(in_file_name)
    source_json = json.load(cyjs_file)
    cyjs_file.close()
    
    

    # Output files
    out = open(out_file_name, "w")

    nodeMap = {}

    sourceNodes = source_json["elements"]["nodes"]
    sourceEdges = source_json["elements"]["edges"]

    min_max = find_min_max(sourceNodes)
    score_range = min_max[1] - min_max[0]
    
    nodeList = []
    edgeList = []

    for node in sourceNodes:
        termID = node["data"]["Manual_Name"]
        x = node["position"]["x"]
        y = node["position"]["y"]
        suid = node["data"]["SUID"]
        if "Size" in node["data"]:
            size = node["data"]["Size"] * 2
        else:
            size = 6.0
        
        if "score" in node["data"]:
            ratio = (node["data"]["score"] - min_max[0]) / score_range
            col1 = Blues(ratio)
            color = (
                int(col1[0] * 255),
                int(col1[1] * 255),
                int(col1[2]* 255)
            )
            print(str(color))
        else:
            color = (200,200,200)
      
    
        
        newNode = {
            'id':termID, 
            'label':termID,
            'size':size,
            'x':x, 'y':y, 
            'color':"rgb(58,150,43)"
        }
        
        nodeMap[str(suid)] = termID
        nodeList.append(newNode)

    for edge in sourceEdges:
        source = nodeMap[edge['data']['source']]
        target = nodeMap[edge['data']['target']]
        interaction = edge['data']['interaction']
        edgeList.append({'source':source, 'target':target, 'relationship':interaction, 'weight':1.0})

    graph = {}

    graph["nodes"] = nodeList
    graph["edges"] = edgeList

    out.write(json.dumps(graph, sort_keys=True, indent=4))
    out.close()

In [ ]:
# Test run
convert2sigma('atgo.cyjs', 'atgo.json')